**<h2>VOICE TO VOICE RESPONSE</h2>**
    <h3>1. VOICE TO TEXT</h3>
    <h3>2. TEXT TO TEXT</h3>
    <h3>3. TEXT TO VOICE</h3>
    
 <h4>Installation of dependencies and models</h4>

In [1]:
!pip install torch torchaudio transformers jiwer
!pip install git+https://github.com/openai/whisper.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 39.3 MB/s eta 0:00:0000:0100:01
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-wp3cvwor
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-wp3cvwor
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=93339ddf51baff3e457b66f62e93581c3cb8dc402746a2ec2d5b04ca349aeb54
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ujfif7p/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully b

<h2>1. VOICE TO TEXT</h2>
<h3>Importing the Installed libraries and model. making a class for Audio transcriber for whiisper model</h3>

In [2]:
import whisper
import torchaudio
import os
import torch

class AudioTranscriber:
    def __init__(self, model_name="base.en"):
        self.model = whisper.load_model(model_name)
    
    def transcribe(self, audio_path):
        audio, sample_rate = torchaudio.load(audio_path)

        if sample_rate != 16000:
            resample = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
            audio = resample(audio)

        audio = whisper.pad_or_trim(audio.flatten()).to("cuda" if torch.cuda.is_available() else "cpu")

        mel = whisper.log_mel_spectrogram(audio)

        options = whisper.DecodingOptions(language="en", without_timestamps=True)
        result = self.model.decode(mel, options)

        return result.text


<h3>Converting Voice into text format and saving it in folder</h3>

In [3]:
audio_file = '/kaggle/input/voices/84-121123-0010.wav'
transcriber = AudioTranscriber()

transcription = transcriber.transcribe(audio_file)
print(f"Transcription:\n{transcription}")

output_dir = "/kaggle/working/output_text"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "transcription.txt")

with open(output_file, "w") as file:
    file.write(transcription)

print(f"Transcription saved to {output_file}")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 136MiB/s]
/opt/conda/lib/python3.10/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Transcription:
Nautier looked upon morale with one of those melancholy smiles which had so often made Valentine happy and thus fixed his attention. Nautier looked upon morale with one of those melancholy smiles which had so often made Valentine happy and thus fixed his attention.
Transcription saved to /kaggle/working/output_text/transcription.txt


<h2>2. TEXT TO TEXT</h2>
<h3>Importing the required transformers and llama-7b model from huggyllama, and preparing response of the text file generated in previous step.</h3>

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextGenerationPipeline

tokenizer = AutoTokenizer.from_pretrained("huggyllama/llama-7b")
model = AutoModelForCausalLM.from_pretrained("huggyllama/llama-7b")

generator = TextGenerationPipeline(model=model, tokenizer=tokenizer)

with open(output_file, "r") as file:
    transcription = file.read()

response = generator(transcription, max_length=150, num_return_sequences=1)
response_text = response[0]['generated_text']
print(f"LLM Response:\n{response_text}")

response_dir = "/kaggle/working/response"
os.makedirs(response_dir, exist_ok=True)
response_file = os.path.join(response_dir, "llm_response.txt")

with open(response_file, "w") as file:
    file.write(response_text)

print(f"LLM Response saved to {response_file}")


tokenizer_config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


LLM Response:
Nautier looked upon morale with one of those melancholy smiles which had so often made Valentine happy and thus fixed his attention. Nautier looked upon morale with one of those melancholy smiles which had so often made Valentine happy and thus fixed his attention.
"I am not a man of the world," he said, "and I have never been in love. I have never been in love. I have never been in love.
"I have never been in love," he said, "and I have never been in love.
"I have never been in love," he said, "and I have never been in love.
"I have never been in love," he
LLM Response saved to /kaggle/working/response/llm_response.txt


<h2>3. TEXT TO VOICE</h2>
<h3>Using `espeak-ng` for voice generation, maing a user dependent function which takes pitch, speed, gender as user parameter and other internal parameters </h3>

In [5]:
!sudo apt-get install espeak-ng -y

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 30 not upgraded.
Need to get 4215 kB of archives.
After this operation, 12.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcaudio0 amd64 1.1-4 [7908 B]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libsonic0 amd64 0.2.0-8 [13.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-6ubuntu0.1 [3682 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-6ubuntu0.1 [190 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 espeak-ng amd64 1.50+dfsg-6ubuntu0.1 [322 kB]


In [10]:
import os
from IPython.display import Audio, display

def text_to_audio_espeak(text, output_file, pitch=50, speed=175, voice='en-us+f2'):
    directory = os.path.dirname(output_file)
    if directory:
        os.makedirs(directory, exist_ok=True)
    
    command = [
        'espeak-ng',
        f'-p {pitch}',   # Set pitch (0 to 99)
        f'-s {speed}',   # Set speed (words per minute)
        f'-v {voice}',   # Set voice, e.g., 'en-us+f2' for a female voice
        f'"{text}"',     # The text to synthesize
        '--stdout'       # Output to stdout for piping to a file
    ]

    os.system(' '.join(command) + f' > {output_file}')
    
    print(f"Audio saved to {output_file}")

<h1>Final conversion of voice by taking input form userand displaying the audio</h1>

In [13]:
gender = str(input("enter gender: "))
output_file = "response.wav"
with open('/kaggle/working/output_text/transcription.txt', 'r') as file:
    text = file.read()

if(gender == 'female'):
    text_to_audio_espeak(text, output_file, pitch=70, speed=150, voice='en-us+f2')
else:
    text_to_audio_espeak(text, output_file, pitch=70, speed=150, voice='en')

display(Audio(output_file, autoplay=True))

enter gender:  male


Audio saved to response.wav
